In [1]:
import pandas as pd
import numpy as np
import time
from datetime import date

In [2]:
df = pd.read_csv('/kaggle/input/cabspotting/new_abboip.txt'
                 ,sep=' '
                 , header=None, names=['lat','long','occupancy','time']
                )

In [3]:
df.dtypes

lat          float64
long         float64
occupancy      int64
time           int64
dtype: object

In [4]:
#1211033530
df['time']=pd.to_datetime(df['time'], utc=True, unit='s' )

In [5]:
size = len(df['occupancy'])

In [6]:
df['trip_no'] = list(range(size))
df['trip_status'] = list(range(size))

curr_trip_no = 1
curr_occupancy = df['occupancy'][0]
df['trip_no'][0] = curr_trip_no
df['trip_status'][0] = 0 # start:0, in-trip:1, end:2

for i in range(1, size-1):
    pre_occupancy = curr_occupancy
    curr_occupancy = df['occupancy'][i]
    if  curr_occupancy == pre_occupancy: # in-trip
        df['trip_no'][i] = curr_trip_no
        df['trip_status'][i] = 1
    else: # trip end
        curr_trip_no = curr_trip_no+1
        df['trip_no'][i] = curr_trip_no
        df['trip_status'][i] = 0
        df['trip_status'][i-1] = 2 ## update the status of previous step to 'end'
# update the last row
df['trip_no'][size-1] = curr_trip_no
df['trip_status'][size-1] = 2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopy

In [7]:
df_start = df.groupby('trip_no').agg({
    'lat':'min',
    'long':'min',
    'time':'min'
}).reset_index().rename(
    columns={
        'lat':'lat_start',
        'long':'long_start',
        'time':'time_start'
    }
)

df_end = df.groupby('trip_no').agg({
    'lat':'max',
    'long':'max',
    'time':'max'
}).reset_index().rename(
    columns={
        'lat':'lat_end',
        'long':'long_end',
        'time':'time_end'
    }
)

In [8]:
df_trip = pd.DataFrame.merge(df_start,df_end, on='trip_no', how='inner')

In [9]:
df_trip

,trip_no,lat_start,long_start,time_start,lat_end,long_end,time_end
0,1,37.74978,-122.39709,2008-05-17 14:12:10+00:00,37.75153,-122.39446,2008-05-17 14:50:44+00:00
1,2,37.74831,-122.41438,2008-05-17 14:51:10+00:00,37.75552,-122.39724,2008-05-17 14:55:58+00:00
2,3,37.75042,-122.42291,2008-05-17 14:56:00+00:00,37.76523,-122.41441,2008-05-17 15:17:55+00:00
3,4,37.75053,-122.43101,2008-05-17 15:18:06+00:00,37.75206,-122.42086,2008-05-17 15:21:48+00:00
4,5,37.74833,-122.43172,2008-05-17 15:21:50+00:00,37.77219,-122.41402,2008-05-17 15:43:15+00:00
...,...,...,...,...,...,...,...
1910,1911,37.75770,-122.43499,2008-06-10 07:11:20+00:00,37.76873,-122.42852,2008-06-10 07:19:00+00:00
1911,1912,37.76873,-122.43719,2008-06-10 07:19:15+00:00,37.77208,-122.42900,2008-06-10 07:22:50+00:00
1912,1913,37.77209,-122.43918,2008-06-10 07:23:21+00:00,37.78719,-122.41689,2008-06-10 07:36:24+00:00
1913,1914,37.78740,-122.41527,2008-06-10 07:36:55+00:00,37.79779,-122.40405,2008-06-10 07:41:55+00:00


In [10]:
df_trip.dtypes

trip_no                     int64
lat_start                 float64
long_start                float64
time_start    datetime64[ns, UTC]
lat_end                   float64
long_end                  float64
time_end      datetime64[ns, UTC]
dtype: object

In [11]:
#df_trip['duration'] = df_trip[['time_start','time_end']].apply(lambda x: pd.Timedelta(x.time_end,x.time_start).seconds )